I will use the MNIST dataset from Sklearn.

In [1]:
from sklearn.datasets import fetch_openml

In [2]:
X, y = fetch_openml("mnist_784", version = 1, return_X_y = True, as_frame=False)

In [38]:
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator
from scipy.sparse import lil_matrix

In [311]:
class SoftmaxLogistic(TransformerMixin, BaseEstimator):
    
    def cost(self, X, y):
        m = X.shape[0]
        cost = 0
        if self.reg == "l1":
            cost += np.sum(np.abs(self.w))*self.reg_weight
        elif self.reg == "l2":
            cost += np.sum(np.square(self.w))*self.reg_weight/2
        
        ypadmul = self.p(X)[np.arange(0, y.shape[0]), y.flatten()]
        cnorm = -np.sum(np.log(ypadmul))/m
        
        return cost + cnorm
        
    def p(self, X):
        s = X @ self.w
        saxmax = np.max(s, axis=1)
        s -= saxmax.reshape(-1, 1)
        sexp = np.exp(s)
        sexpsum = np.sum(sexp, axis=1)
        smax = sexp/sexpsum.reshape(-1,1)
        return smax
    
    def dw(self, X, y):
        m = X.shape[0]
        p_x = self.p(X)
        diff = p_x 
        diff[np.arange(0, y.shape[0]), y.flatten()] -= 1
        return X.T @ diff / m
        
    
    def __init__(self, reg = None, reg_weight = 1.0, learning_rate = 1e-10):
        self.reg = reg
        self.reg_weight = reg_weight
        self.learning_rate = learning_rate
        
        if reg is None:
            self.d_reg = lambda : 0
        elif reg == "l1":
            self.d_reg = lambda w : reg_weight * np.sign(w[1:,:])
        elif reg == "l2":
            self.d_reg = lambda w : reg_weight * w[1:,:]
            
    def encode_y(self, y):
        ny = np.zeros_like(y)
        for index, value in enumerate(y):
            if value in self.classes:
                ny[index] = self.classes[value]
        return ny.astype(int)
    
    def predict(self, X):
        X_biased = np.c_[np.ones((X.shape[0], 1)), X]
        X_p = self.p(X_biased)
        y_idx = np.argmax(X_p, axis=1)
        y = np.zeros((X.shape[0], 1), dtype=object)
        for index, key in enumerate(y_idx):
            y[index] = self.keys[key]
        return y
    
    def fit(self, X, y, verbose = 0, delta_cost = 1e-5):
        X_biased = np.c_[np.ones((X.shape[0],1)), X]
        
        #number of classes
        self.classes = {}
        self.keys = {}
        for value in y:
            if value not in self.classes:
                self.classes[value] = len(self.classes)
                self.keys[self.classes[value]] = value
        self.nclasses = len(self.classes)
        
        y_enc = self.encode_y(y)
                
        self.w = np.zeros(shape=(X_biased.shape[1], self.nclasses))
        cost = float("inf")
        if verbose > 0:
            print("Training starts")
        while True:
            newcost = self.cost(X_biased, y_enc)
            if abs(newcost - cost) < delta_cost:
                break
            cost = newcost
            if verbose > 0:
                print("Cost: ", self.cost(X_biased, y_enc), " " * 20, end="\r")
            dw = self.dw(X_biased, y_enc)
            self.w = self.w - self.learning_rate*dw
        if verbose > 0:
            print("Training done with delta", delta_cost)
        return self

In [312]:
smax = SoftmaxLogistic(learning_rate=0.00001, reg="l2")

In [313]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [315]:
smax.fit(X_train, y_train, verbose=1, delta_cost=1e-4)

Training starts
Training done with delta 0.0001                


In [319]:
result = smax.predict(X_test)

In [320]:
from sklearn.metrics import accuracy_score

In [321]:
accuracy_score(y_test, result)

0.9094285714285715

Quite good of a result for a first try. I don't really want or need to tamper with the hyperparameters, but I will now implement the early stopping version and compare the result, but first I will compare the result with softmax from sklearn.

In [326]:
from sklearn.linear_model import LogisticRegression

In [329]:
lib_linreg = LogisticRegression(penalty="l2", C=1.0, multi_class="multinomial", max_iter=2000)

In [330]:
lib_linreg.fit(X_train, y_train)

/home/karolszustakowski/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000, multi_class='multinomial')

2000 iterations is way more than my algorithm used, so I will not train more.

In [331]:
pred_lib = lib_linreg.predict(X_test)

In [332]:
accuracy_score(pred_lib, y_test)

0.9115428571428571

Honestly, this is not much better. Let's see what I can get with early stopping.

In [489]:
import random
from sklearn.utils import shuffle

class SoftmaxEarlyStopping(BaseEstimator, TransformerMixin):
    
    def __init__(self, learning_rate = 1e-4):
        self.learning_rate = 1e-4
    
    def encode_y(self, y):
        ny = np.zeros_like(y)
        for index, value in enumerate(y):
            if value in self.classes:
                ny[index] = self.classes[value]
        return ny.astype(int)
    
    def p(self, X):
        s = X @ self.w
        smax = np.amax(s, axis=1).reshape(-1, 1)
        s = s - smax
        sexp = np.exp(s)
        ssum = np.sum(sexp, axis=1).reshape(-1, 1)
        return sexp/ssum
    
    def cost(self, X, y):
        m = X.shape[0]
        p_x = self.p(X)
        c_x = p_x[np.arange(0, y.shape[0]), y.flatten()]
        return - np.sum(np.log(c_x)) / m
        
    def predict(self, X, add_bias = True, encode = True):
        if add_bias:
            X_b = np.c_[np.ones((X.shape[0], 1)), X]
        else:
            X_b = X
        p = self.p(X)
        y_idx = np.argmax(p, axis = 1)
        if encode:
            y = np.zeros(y_idx.shape, dtype=object)
            for i, index in enumerate(y_idx):
                y[i] = self.keys[index]
        else:
            y = y_idx
        return y
    
    def dw(self, X, y):
        m = X.shape[0]
        p_x = self.p(X)
        p_x[np.arange(0, y.shape[0]), y.flatten()] -= 1
        return X.T @ p_x / m
    
    def learning_schedule(self, epoch, decay = 0.005):
        return self.learning_rate/(1 + decay*epoch)
    
    def fit(self, X, y, verbose = 0, n_violations = 20, batch_size = 1000, delta_cost = 1e-5):
        #split the dataset into training and validation
        X_biased = np.c_[np.ones((X.shape[0], 1)), X]
                
        self.classes = {}
        self.keys = {}
        
        for c in y:
            if c not in self.classes:
                n = len(self.classes)
                self.classes[c] = n
                self.keys[n] = c
        
        self.nclasses = len(self.classes)
        y_enc = self.encode_y(y)
        X, y_enc = shuffle(X, y_enc)
        
        X_b_train, X_b_val, y_train, y_val = train_test_split(X, y_enc, test_size=0.2)
        self.w = np.zeros((X_b_train.shape[1], self.nclasses))        
        cost = float("inf")
        minCost = float("inf")
        n_voil = 0
        best_w = self.w.copy()
        
        epoch = 0
        
        if verbose > 0:
            print("Starting training")
            
        while True:
            lr = self.learning_schedule(epoch)
            
            epoch += 1
            val_cost = self.cost(X_b_val, y_val)
                        
            sample = random.sample(range(X_b_train.shape[0]), batch_size)
                    
            train_part_X = X_b_train[sample, :]
            train_part_y = y_train[sample]
            
            self.w = self.w - self.dw(train_part_X, train_part_y)*lr
            
            val_cost = self.cost(X_b_val, y_val)
            if abs(val_cost - cost) < delta_cost:
                if verbose > 0:
                    print("Delta cost achieved. Stopping the training")
                break
            
            cost = val_cost
            
            if verbose > 0:
                val_pred = self.predict(X_b_val, add_bias=False, encode = False)
                acc = accuracy_score(val_pred, y_val)
                print("Val score: {}, best score: {}, val acc: {}".format(val_cost, minCost, acc), " "*50, end="\r")
            
            if val_cost < minCost:
                minCost = val_cost
                n_voil = 0
                best_w = self.w.copy()
            else:
                n_voil += 1
                
            if n_voil > n_violations:
                self.w = best_w
                if verbose > 0:
                    print("\nVal error rising, selecting best model")
                break
        if verbose > 0:
            print("\nTraining done with validation error score:", minCost)   
            print("\nFinal validation accuracy:")
            val_pred = self.predict(X_b_val, add_bias=False, encode = False)
            acc = accuracy_score(val_pred, y_val)
            print(acc)
        return self

In [490]:
sme = SoftmaxEarlyStopping(learning_rate=0.000001)

In [491]:
sme.fit(X_train, y_train, verbose=1, n_violations=40)

Starting training
Val score: 0.30513130911670894, best score: 0.29816303996725174, val acc: 0.9158095238095239                                                                                                                                                                                                          
Val error rising, selecting best model

Training done with validation error score: 0.29816303996725174

Final validation accuracy:
0.92


SoftmaxEarlyStopping()

Let's now evaluate this model on the test set!

In [492]:
test_pred_estop = sme.predict(X_test)

In [495]:
accuracy_score(test_pred_estop, y_test)

0.9178285714285714